In [1]:
import pandas as pd
import numpy as np

from scipy import stats
from sympy import *
from statsmodels.stats.proportion import proportions_ztest

import matplotlib.pyplot as plt

pd.options.display.float_format = '{:,.4f}'.format

import warnings
warnings.filterwarnings("ignore")

# Критерий Манна-Уитни

$$U = (n_1 \cdot n_2) + \frac{n_{max} \cdot (n_{max} + 1)}{2} - T$$

## Задача 1

При измерении пространственных порогов тактильной чувствительности получены следующие величины порогов тактильной чувствительности 

|Муж|Жен|
|---|---|
|39|32|
|36|30|
|31|28|
|35|30|
|29|33|
|34|37|
|38|28|
|-|27|

Отличаются ли между собой пороги мужчин и женщин?

$$H_0: P(X>Y) = P(X<Y)$$
$$H_1: P(X>Y) \neq P(X<Y)$$

In [81]:
men = pd.Series([39, 36, 31, 35, 29, 34, 38])
women = pd.Series([32, 30, 28, 30, 33, 37, 28, 27])

df = pd.DataFrame({'Муж':men, 'Жен':women})

df

,Муж,Жен
0,39.0000,32
1,36.0000,30
2,31.0000,28
3,35.0000,30
4,29.0000,33
5,34.0000,37
6,38.0000,28
7,nan,27


#### Расчет вручную 

In [82]:
T = pd.concat([df['Муж'], df['Жен']])
T = T.sort_values().dropna()
rang = stats.rankdata(T)

rang_df = pd.DataFrame({'value':T, 'rang':rang})

rang_df = rang_df.drop_duplicates(subset=['value'])

In [83]:
merge = df.merge(rang_df, how='left', left_on=['Муж'], right_on=['value'])\
          .merge(rang_df, how='left', left_on=['Жен'], right_on=['value'])

merge = merge.drop(['value_x', 'value_y'], axis=1)

merge

,Муж,Жен,rang_x,rang_y
0,39.0000,32,15.0000,8.0000
1,36.0000,30,12.0000,5.5000
2,31.0000,28,7.0000,2.5000
3,35.0000,30,11.0000,5.5000
4,29.0000,33,4.0000,9.0000
5,34.0000,37,10.0000,13.0000
6,38.0000,28,14.0000,2.5000
7,nan,27,nan,1.0000


In [84]:
n_1, n_2 = merge['Муж'].count(), merge['Жен'].count()

val_max = max(merge.rang_x.sum(), merge.rang_y.sum())

n_max = np.nan

if merge.rang_x.sum() == val_max:
    n_max = df['Муж'].count()
elif merge.rang_y.sum() == val_max:
    n_max = df['Жен'].count()
    
n_max

7

In [87]:
u = (n_1 * n_2) + (n_max * (n_max + 1)) / 2 - val_max

u

11.0

Для критерия Манна-Уитни при $n_1=7$, $n_2=8$ критическое значение равно $13$. Так как наше значение статистики меньше критичекого (попадает в критическую область), следовательно мы **отвергаем нулевую гипотезу** и говорим, что **пороги мужчин и женщин отличаются между собой**. 

#### Расчет через пакет stats

In [88]:
stats.mannwhitneyu(df['Муж'], df['Жен'], nan_policy='omit', alternative='two-sided')

MannwhitneyuResult(statistic=45.0, pvalue=0.055757917688854436)

In [52]:
from pingouin import mwu

mwu(df['Муж'], df['Жен'], alternative='two-sided')

,U-val,alternative,p-val,RBC,CLES
MWU,45.0000,two-sided,0.0558,-0.6071,0.8036


## Задача 2

По выборкам из 2-х партий микросхем после операции легирования поликремния измерялось удельное сопротивление. Результаты замеров следующие. 

||1|2|3|4|5|6|7|8|9|10|
|---|---|---|---|---|---|---|---|---|---|---|
|1-я партия|52.2|33|32.5|49.5|32.5|191.5|112.5|69.1|48.5|16.5|
|2-я партия|119|17.5|43.5|90.5|40.0|50.0|108|96|-|-|

Отличаются ли удельные сопротивления? 

$$H_0: P(X>Y) = P(X<Y)$$
$$H_1: P(X>Y) \neq P(X<Y)$$

In [73]:
part_1 = pd.Series([52.2, 33, 32.5, 49.5, 32.5, 191.5, 112.5, 69.1, 48.5, 16.5])
part_2 = pd.Series([119, 17.5, 43.5, 90.5, 40.0, 50.0, 108, 96])

df = pd.DataFrame({'part_1':part_1, 'part_2':part_2})
df

,part_1,part_2
0,52.2000,119.0000
1,33.0000,17.5000
2,32.5000,43.5000
3,49.5000,90.5000
4,32.5000,40.0000
5,191.5000,50.0000
6,112.5000,108.0000
7,69.1000,96.0000
8,48.5000,nan
9,16.5000,nan


#### Расчет вручную 

In [74]:
T = pd.concat([df['part_1'], df['part_2']])
T = T.sort_values().dropna()
rang = stats.rankdata(T)

rang_df = pd.DataFrame({'value':T, 'rang':rang})

rang_df = rang_df.drop_duplicates(subset=['value'])

In [75]:
merge = df.merge(rang_df, how='left', left_on=['part_1'], right_on=['value'])\
          .merge(rang_df, how='left', left_on=['part_2'], right_on=['value'])

merge = merge.drop(['value_x', 'value_y'], axis=1)

merge

,part_1,part_2,rang_x,rang_y
0,52.2000,119.0000,11.0000,17.0000
1,33.0000,17.5000,5.0000,2.0000
2,32.5000,43.5000,3.5000,7.0000
3,49.5000,90.5000,9.0000,13.0000
4,32.5000,40.0000,3.5000,6.0000
5,191.5000,50.0000,18.0000,10.0000
6,112.5000,108.0000,16.0000,15.0000
7,69.1000,96.0000,12.0000,14.0000
8,48.5000,nan,8.0000,nan
9,16.5000,nan,1.0000,nan


In [76]:
n_1, n_2 = merge['part_1'].count(), merge['part_2'].count()

val_max = max(merge.rang_x.sum(), merge.rang_y.sum())

n_max = np.nan

if merge.rang_x.sum() == val_max:
    n_max = df['part_1'].count()
elif merge.rang_y.sum() == val_max:
    n_max = df['part_2'].count()
    
n_max

10

In [77]:
u = (n_1 * n_2) + (n_max * (n_max + 1)) / 2 - val_max

u

48.0

In [78]:
print(n_1, n_2)

10 8


$T_{крит}$ для $n_1=10$ и $n_2=8$ равен $17$. Значение нашей статистики критерия $U=48$, что больше $T_{крит}$. Следовательно в критическую область не попадаем, **отвергнуть нулевую гипотезу не можем. Различий не существует**.

#### Расчет пакетом stats

In [79]:
stats.mannwhitneyu(df['part_1'], df['part_2'], nan_policy='omit', alternative='two-sided')

MannwhitneyuResult(statistic=32.0, pvalue=0.5049403898598952)

# Критерий Уилкоксона

$$T_{эмп} = \sum{R_r}$$

, где $\sum{R_r}$ - сумма нетипичных (более редких) рангов

|X|Y|
|---|---|
|13|16|
|15|14|
|12|14|
|11|12|
|15|11|
|13|15|

$$H_0: (X - Y) = 0$$
$$H_1: (X - Y) \neq 0$$

In [32]:
X = [13, 15, 12, 11, 15, 13]
Y = [16, 14, 14, 12, 11, 15]

df = pd.DataFrame({'X':X, 'Y':Y})

df['X-Y'] = df['X'] - df['Y']

df['abs'] = abs(df['X-Y'])

df['rang'] = stats.rankdata(df['abs'])

df

,X,Y,X-Y,abs,rang
0,13,16,-3,3,5.0000
1,15,14,1,1,1.5000
2,12,14,-2,2,3.5000
3,11,12,-1,1,1.5000
4,15,11,4,4,6.0000
5,13,15,-2,2,3.5000


#### Рачсет вручную

In [33]:
below_zero = df[df['X-Y']<=0]['X'].count()
above_zero = df[df['X-Y']>0]['X'].count()


print(f'below zero: {below_zero}, above zero: {above_zero}')

below zero: 4, above zero: 2


In [43]:
sum_rang = df[df['X-Y']>0]['rang'].sum()
sum_rang

7.5

Значение критерия Уилкоксона выше критического значиния для $n=6$ ($T_{крит}=2$). Следовательно, не можем отвергнуть нулевую гипотезу. **Различий не существует**. 

#### Расчет пакетом stats

In [42]:
stats.wilcoxon(df['X'], df['Y'])

WilcoxonResult(statistic=7.5, pvalue=0.5625)

$p-value$ выше $\alpha=0.05$. Следовательно, не можем отвергнуть нулевую гипотезу. **Различий не существует**. 

## Задача

Дан уровень тревожности ло и после тренинга

|№|Уровень тревожности (до тренинга)|Уровень тревожности (после тренинга)|
|---|---|---|
|1|15|14|
|2|14|11|
|3|16|17|
|4|18|19|
|5|21|20|
|6|21|18|
|7|20|15|
|8|15|17|
|9|17|14|
|10|13|12|

Проверить на уровне статистической значимости $\alpha = 0.05$ снизился ли уровень тревожности после тренинга


$$H_0: (X - Y) = 0$$
$$H_1: (X - Y) > 0$$

In [90]:
X = [15, 14, 16, 18, 21, 21, 20, 15, 17, 13]
Y = [14, 11, 17, 19, 20, 18, 15, 17, 14, 12]

df = pd.DataFrame({'X':X, 'Y':Y})

df.index = range(1, len(df)+1)

df['X-Y'] = df['X'] - df['Y']

df['abs'] = abs(df['X-Y'])

df['rang'] = stats.rankdata(df['abs'])

df

,X,Y,X-Y,abs,rang
1,15,14,1,1,3.0000
2,14,11,3,3,8.0000
3,16,17,-1,1,3.0000
4,18,19,-1,1,3.0000
5,21,20,1,1,3.0000
6,21,18,3,3,8.0000
7,20,15,5,5,10.0000
8,15,17,-2,2,6.0000
9,17,14,3,3,8.0000
10,13,12,1,1,3.0000


#### Расчет вручную

In [91]:
below_zero = df[df['X-Y']<=0]['X'].count()
above_zero = df[df['X-Y']>0]['X'].count()


print(f'below zero: {below_zero}, above zero: {above_zero}')

below zero: 3, above zero: 7


In [93]:
sum_rang = df[df['X-Y']<=0]['rang'].sum()
sum_rang

12.0

In [98]:
sum_rang = df[df['X-Y']>0]['rang'].sum()
sum_rang

43.0

Значение критерия Уилкоксона выше критического значиния для $n=10$ ($T_{крит}=10$). Следовательно, не можем отвергнуть нулевую гипотезу. **Уровень тревожности после тренинга не снизился**. 

#### Расчет пакетом stats

In [102]:
stats.wilcoxon(df['X'], df['Y'], alternative='greater')

WilcoxonResult(statistic=43.0, pvalue=0.0654296875)

$p-value$ выше $\alpha=0.05$. Следовательно, не можем отвергнуть нулевую гипотезу. **Уровень тревожности после тренинга не снизился**. 